### SU(3) matrix exponential
Taylor series, improved Taylor series and Cayley-Hamilton.

In [1]:
su_group = 'su3'

In [2]:
import os
os.environ["MY_NUMBA_TARGET"] = "python"
os.environ["PRECISION"] = "double"
os.environ["GAUGE_GROUP"] = su_group

import sys
sys.path.append('..')

import curraun.su as su
Nc = su.NC

import numpy as np
from scipy.linalg import expm

Using SU(3)
Using Python
Using double precision


In [3]:
# Normalize determinant to 1
def normalize(U):
    detU = np.linalg.det(U)
    U = U / (detU ** (1/Nc))
    return U

# Reunitarize using SVD: U = V @ W^\dagger where U = V S W^\dagger
def unitarize(U):
    V, S, Wdag = np.linalg.svd(U)  
    Uunit = V @ Wdag              
    return Uunit

# Check unitarit
def check_unitarity(U):
    id = np.eye(Nc, dtype=U.dtype)
    UdagU = U.conj().T @ U
    unit_err = np.linalg.norm(UdagU - id)
    return np.allclose(UdagU, id)
    # return unit_err

In [4]:
def generate_su3_matrix(seed, epsilon=0.1):
    """Generate a random SU(3) matrix close to identity (group element)"""
    np.random.seed(seed)
    H = np.random.randn(Nc, Nc) + 1j * np.random.randn(Nc, Nc) # random complex SU(3) matrix 
    H = (H + H.conj().T) / 2  # make it Hermitian
    H = H - np.trace(H) / Nc * np.eye(Nc)  # make it traceless

    generator = 1j * epsilon * H # anti-Hermitian generator
    Ugen = np.eye(Nc, dtype=np.complex128) + generator # matrix close to identity
    # U = expm(generator) # matrix close to identity

    Uunit = unitarize(Ugen) # reunitarize
    U = normalize(Uunit) # normalize determinant
    
    Uflat = U.reshape(Nc*Nc)    

    print(f"Determinant: {np.linalg.det(U)}")
    print(f"Unitary: {check_unitarity(U)}")

    return Uflat

In [5]:
def generate_su3_generator(seed, epsilon=0.1):
    """Generate an anti-Hermitian, traceless generator (algebra element)"""
    np.random.seed(seed)
    H = np.random.randn(Nc, Nc) + 1j * np.random.randn(Nc, Nc) 
    H = (H + H.conj().T) / 2  # make it Hermitian
    H = H - np.trace(H) / Nc * np.eye(Nc)  # make it traceless

    gen = 1j * epsilon * H  # anti-Hermitian generator
    gflat = gen.flatten()

    print(f"Generator trace: {np.trace(gen)}")
    print(f"Anti-Hermitian: {np.allclose(gen, -gen.conj().T)}")

    return gflat

In [6]:
seed = 314
U = generate_su3_matrix(seed, epsilon=0.1)
U

Determinant: (1.0000000000000004+2.4286128663675305e-17j)
Unitary: True


array([ 0.99704112+0.06747425j, -0.01239478+0.00291967j,
        0.01437942+0.03142135j,  0.00917845+0.00451592j,
        0.99438981-0.04152151j,  0.07534223+0.06069421j,
       -0.01671446+0.03118033j, -0.07103392+0.06524495j,
        0.99437611-0.02572623j])

In [7]:
g = generate_su3_generator(seed, epsilon=0.1)
# g = su.unit()
g

Generator trace: -6.938893903907228e-18j
Anti-Hermitian: True


array([ 0.        +0.06788012j, -0.01086386+0.00374468j,
        0.01565808+0.03152477j,  0.01086386+0.00374468j,
       -0.        -0.04189415j,  0.07371073+0.06341922j,
       -0.01565808+0.03152477j, -0.07371073+0.06341922j,
       -0.        -0.02598597j])

In [8]:
# mexp = su.mexp(U, EXP_MAX_TERMS=10, EXP_ACCURACY_SQUARED=1e-32) # SU(3) matrix exponential 
mexp = su.mexp(g)
mexp

((0.9970129001406108+0.06779453787960968j),
 (-0.012461405973638534+0.0029299732680562567j),
 (0.01444198917438123+0.031571112641057185j),
 (0.009214392674363641+0.004541460785403434j),
 (0.9943363606681792-0.04171747533075365j),
 (0.07570930540860747+0.06097069198649506j),
 (-0.01679934895756539+0.031327776222991745j),
 (-0.07135997907640178+0.06556474745785822j),
 (0.9943225262732092-0.02584730806467606j))

In [9]:
# mexp_improved = su.mexp_improved(g, exp_quality=1e2, min_n=4)
mexp_improved = su.mexp_improved(g)
mexp_improved

((0.9971059162124398+0.06780238654097549j),
 (-0.012413324587002107+0.0029558662862404455j),
 (0.014482134902203467+0.03157419437622305j),
 (0.009267235820845125+0.004517214878131249j),
 (0.9945122462188761-0.04173366114283106j),
 (0.07565804033587903+0.0610557523640876j),
 (-0.016766061891664265+0.03133850719590927j),
 (-0.07144351912665911+0.06550745496673603j),
 (0.9944988861148661-0.025860010007160438j))

In [10]:
mexp_cayham = su.mexp_cayham(g)
mexp_cayham

((0.997012900140611+0.06779453787960968j),
 (-0.012461405973638706+0.0029299732680561352j),
 (0.014441989174380998+0.03157111264105724j),
 (0.009214392674363414+0.004541460785403573j),
 (0.9943363606681792-0.041717475330753664j),
 (0.07570930540860754+0.06097069198649502j),
 (-0.01679934895756563+0.03132777622299168j),
 (-0.07135997907640179+0.06556474745785829j),
 (0.994322526273209-0.02584730806467607j))

In [11]:
mexp_scipy = expm(g.reshape(Nc, Nc)).reshape(Nc*Nc)
mexp_scipy

array([ 0.9970129 +0.06779454j, -0.01246141+0.00292997j,
        0.01444199+0.03157111j,  0.00921439+0.00454146j,
        0.99433636-0.04171748j,  0.07570931+0.06097069j,
       -0.01679935+0.03132778j, -0.07135998+0.06556475j,
        0.99432253-0.02584731j])

In [12]:
print("Comparison of matrix exponential methods:")
print(f"Taylor (mexp):            {mexp[0]}")
print(f"Improved (mexp_improved): {mexp_improved[0]}")
print(f"Cayley-Hamilton:          {mexp_cayham[0]}")
print(f"SciPy (expm):             {mexp_scipy[0]}")
print()

# Check differences from scipy (assuming it's the most accurate)
scipy_val = complex(mexp_scipy[0])  
print("Errors relative to SciPy expm:")
print(f"Taylor error:           {abs(mexp[0] - scipy_val):.2e}")
print(f"Improved Taylor error:  {abs(mexp_improved[0] - scipy_val):.2e}")
print(f"Cayley-Hamilton error:  {abs(mexp_cayham[0] - scipy_val):.2e}")

Comparison of matrix exponential methods:
Taylor (mexp):            (0.9970129001406108+0.06779453787960968j)
Improved (mexp_improved): (0.9971059162124398+0.06780238654097549j)
Cayley-Hamilton:          (0.997012900140611+0.06779453787960968j)
SciPy (expm):             (0.9970129001406108+0.06779453787960968j)

Errors relative to SciPy expm:
Taylor error:           0.00e+00
Improved Taylor error:  9.33e-05
Cayley-Hamilton error:  1.11e-16
